## Select dùng để làm gì?

Không có select, một lúc chúng ta chỉ có thể thao tác với 1 channel mà thôi

In [ ]:
msg1 := <-ch1  // Block cho tới khi ch1 có data
msg2 := <-ch2  // Rồi sau đó mới check ch2

Có select thì có thể sử dụng nhiều channel đồng thời

In [ ]:
select {
case msg1 := <-ch1:
    fmt.Println("Got message from ch1:", msg1)
case msg2 := <-ch2:
    fmt.Println("Got message from ch2:", msg2)
}

## Sử dụng Select

### Bắt đầu đơn giản với 1 case

Lúc này thì cả đoạn select cũng chỉ tương đương với `msg := <-ch` mà thôi, nhưng để select sẵn

In [3]:
func main() {
    ch := make(chan string)
    
    // Start a goroutine that sends after 1 second
    go func() {
        time.Sleep(1 * time.Second)
        ch <- "Hello!"
    }()
    
    // Select with single case
    select {
    case msg := <-ch:
        fmt.Println("Received:", msg)
    }
}

Received: Hello!


### Với nhiều channels
- Chúng ta có 2 goroutines, 1 cái 1s xử lí xong và 1 cái 2s mới xử lí xong
- Trường hợp này, main sẽ in ra cái xử lí nhanh hơn

In [5]:
func main() {
    ch1 := make(chan string)
    ch2 := make(chan string)
    
    // Two goroutines sending at different times
    go func() {
        time.Sleep(1 * time.Second)
        ch1 <- "one"
    }()
    
    go func() {
        time.Sleep(2 * time.Second)
        ch2 <- "two"
    }()
    
    // Select waits for the first one
    select {
    case msg1 := <-ch1:
        fmt.Println("ch1:", msg1)
    case msg2 := <-ch2:
        fmt.Println("ch2:", msg2)
    }
    // Output: ch1: one (after 1 second)
}

ch1: one


### The Default Case
Để không bị blocking, cần thêm default cho select

In [7]:
func main() {
    ch := make(chan int)
    
    select {
    case val := <-ch:
        fmt.Println("Received:", val)
    default:
        fmt.Println("No value ready")
    }
    // Output: No value ready
}

No value ready


Ví dụ như check mail:

In [8]:
func checkMail(mailbox <-chan string) {
    select {
    case mail := <-mailbox:
        fmt.Println("New mail:", mail)
    default:
        fmt.Println("No new mail")
    }
}

## Basic Timeout Pattern
Nếu goroutine xử lí dưới 2s, sẽ in ra result, còn 2s trở lên sẽ in ra Timeout!

In [15]:
func main() {
    ch := make(chan string)
    
    go func() {
        time.Sleep(3 * time.Second) // change 1, 2 3s to see
        ch <- "data"
    }()
    
    select {
    case result := <-ch:
        fmt.Println("Got result:", result)
    case <-time.After(2 * time.Second):
        fmt.Println("Timeout!")
    }
    // Output: Timeout! (after 2 seconds)
}

Timeout!


Ở ví dú trên chúng ta cũng thấy rằng time.After cũng là 1 channel   
Func signature là: `func After(d Duration) <-chan Time`, có input là 1 duration và trả về 1 receive only channel

In [17]:
func main() {
    ch := time.After(2 * time.Second)
    fmt.Printf("%T\n", ch)   // <-chan time.Time

    fmt.Println(<-ch)        // blocks until 2 seconds pass, then prints the timestamp
}

<-chan time.Time
2025-09-09 07:58:46.608145006 +0000 UTC m=+2.000833343


## Send/Receive với Select
Select có thể được sử dụng cho cả send và receive data tới channel

### Một ví dụ đơn giản với send

In [19]:
func main() {
    ch := make(chan int, 1) // Buffered
    
    select {
    case ch <- 42:
        fmt.Println("Sent value")
    default:
        fmt.Println("Channel full")
    }
    
    // Try again when full
    select {
    case ch <- 43:
        fmt.Println("Sent another")
    default:
        fmt.Println("Still full")
    }
}

Sent value
Still full


- Chúng ta tạo 1 channel với buffered là 1, (đọc phần 02 để hiểu về buffered và unbuffered channel)
- Ở Select trên, lúc này buffer còn empty, nên chúng ta có thể send 42 vào channel, đoạn code sẽ in ra "Send value"
- Ở Select bên dưới, lúc này buffer đã full, nên không thể send 43 vào channel được nữa, nên đoạn code sẽ in ra ở phía default là "Still full"
- Nếu đây không phải là 1 buffered channel, thì không thể send 42 vào channel mà ko có receive, nên đoạn select trên sẽ in ra "Channel full", select bên dưới thì tương tự, xem đoạn code ở dưới:

In [21]:
func main() {
    ch := make(chan int) // Unbuffered
    
    select {
    case ch <- 42:
        fmt.Println("Sent value")
    default:
        fmt.Println("Channel full")
    }
    
    // Try again when full
    select {
    case ch <- 43:
        fmt.Println("Sent another")
    default:
        fmt.Println("Still full")
    }
}

Channel full
Still full


### Một ví dụ đơn giản với receive dùng vòng lặp
- Chúng ta có 2 goroutines, một goroutine xử lí trong 700s và gửi data tới ch1, và một goroutine xử lí trong 300s và send data tới ch2
- Reeceive chạy trong vòng lặp 

In [25]:
func main() {
    ch1 := make(chan int)
    ch2 := make(chan int)
    
    // Send values periodically
    go func() {
        for i := 1; i <= 5; i++ {
            ch1 <- i
            time.Sleep(700 * time.Millisecond)
        }
    }()
    
    go func() {
        for i := 1; i <= 5; i++ {
            ch2 <- i * 10
            time.Sleep(300 * time.Millisecond)
        }
    }()
    
    // Receive from both
    for i := 0; i < 10; i++ {
        select {
        case v1 := <-ch1:
            fmt.Println("ch1:", v1)
        case v2 := <-ch2:
            fmt.Println("ch2:", v2)
        }
    }
}

ch2: 10
ch1: 1
ch2: 20
ch2: 30
ch1: 2
ch2: 40
ch2: 50
ch1: 3
ch1: 4
ch1: 5


#### Một ví dụ khác:

In [26]:
func main() {
    ch := make(chan string, 2) // small buffer

    // Consumer: reads slowly
    go func() {
        for msg := range ch {
            fmt.Println("Consumed:", msg)
            time.Sleep(500 * time.Millisecond)
        }
    }()

    // Producer: generates quickly
    for i := 1; i <= 5; i++ {
        logMsg := fmt.Sprintf("log-%d", i)

        select {
        case ch <- logMsg:
            fmt.Println("Sent:", logMsg)
        default:
            fmt.Println("Dropped:", logMsg)
        }

        time.Sleep(100 * time.Millisecond) // producer is fast
    }

    time.Sleep(3 * time.Second)
}

Sent: log-1
Consumed: log-1
Sent: log-2
Sent: log-3
Dropped: log-4
Dropped: log-5
Consumed: log-2
Consumed: log-3


- Đây là pattern cơ bản Producer/Consumer
- Chúng ta có 1 channel có buffer là 2, buffer tương đối nhỏ
- Consumer tiêu thụ data chậm, consume mỗi data trong 500ms
- Nhưng Producer thì lại tạo data nhanh, mỗi data trong 100s
- Khi khởi tạo, Producer tạo ta log-1 và Consumer cũng tiêu thụ
- Nhưng mỗi 100ms, Producer tạo log-2, log-3, log-4, log-5, lúc này:
    - Sau khi tạo log-2, log-3 thì buffer đầy, nên log-4, log-5 bị drop
    - Sau 500s sau đó, Consumer mới consume log-2
    - Và phải thêm 500s nữa, thì Consumer mới consuem log-3
    - log-4 và log-5 không thể đến được Consumer
- Để Consumer có thể consume hết data thì:
    - Tăng buffer
    - Giảm thời gian consume của Consumer
    - Tăng thời gian produce của Producer

## Vài Behavior của Select

### 1. Nếu data của channel đến song song thì Select sẽ chọn random

In [29]:
func main() {
    ch1 := make(chan string, 1)
    ch2 := make(chan string, 1)
    
    // Both channels have data
    ch1 <- "first"
    ch2 <- "second"
    
    // Select chooses randomly!
    select {
    case msg := <-ch1:
        fmt.Println("Got:", msg)
    case msg := <-ch2:
        fmt.Println("Got:", msg)
    }
    // Could be either "first" or "second"
}

Got: first


In [33]:
func main() {
    ch1 := make(chan string, 1)
    ch2 := make(chan string, 1)
    
    // Both channels have data
    ch1 <- "first"
    ch2 <- "second"
    
    // Select chooses randomly!
    select {
    case msg := <-ch1:
        fmt.Println("Got:", msg)
    case msg := <-ch2:
        fmt.Println("Got:", msg)
    }
    // Could be either "first" or "second"
}

Got: second


### 2. Không có default sẽ bị block

In [34]:
func main() {
    ch1 := make(chan int)
    ch2 := make(chan int)
    
    // This will block forever!
    select {
    case <-ch1:
        fmt.Println("ch1")
    case <-ch2:
        fmt.Println("ch2")
    }
    // Never reaches here
}

fatal error: all goroutines are asleep - deadlock!

goroutine 1 [select]:
main.main()
	 [[ Cell [34] Line 6 ]] /tmp/gonb_11f3669a/main.go:22 +0x71
exit status 2


### 3. Select có thể detect channel đã được close

In [35]:
func main() {
    ch := make(chan int, 2)
    ch <- 1
    ch <- 2
    close(ch)
    
    for i := 0; i < 4; i++ {
        select {
        case val, ok := <-ch:
            if ok {
                fmt.Println("Value:", val)
            } else {
                fmt.Println("Channel closed")
            }
        }
    }
}
// Output:
// Value: 1
// Value: 2
// Channel closed
// Channel closed

Value: 1
Value: 2
Channel closed
Channel closed


## Vài ví dụ:

### 1. Non-blocking Send

In [36]:
func trySend(ch chan<- int, value int) bool {
    select {
    case ch <- value:
        return true
    default:
        return false
    }
}

### 2. First Response

In [37]:
func firstResponse(urls ...string) string {
    ch := make(chan string, len(urls))
    
    for _, url := range urls {
        go func(u string) {
            // Simulate fetch
            time.Sleep(time.Duration(rand.Intn(3)) * time.Second)
            ch <- fmt.Sprintf("Response from %s", u)
        }(url)
    }
    
    return <-ch // First one wins
}

### 3. Ticker with Stop

In [38]:
func ticker(stop <-chan bool) {
    tick := time.Tick(1 * time.Second)
    
    for {
        select {
        case <-tick:
            fmt.Println("Tick")
        case <-stop:
            fmt.Println("Stopped")
            return
        }
    }
}

## Các lỗi thường gặp

### 1. Unnecessary Select
Nếu chỉ là single channel thì receive luôn chứ ko cần dùng Select

In [ ]:
// Don't use select for single channel
select {
case msg := <-ch:
    process(msg)
}

// ✅ Just receive directly
msg := <-ch
process(msg)

### 2. Busy Waiting

In [ ]:
// CPU intensive
for {
    select {
    case msg := <-ch:
        process(msg)
    default:
        // Spins constantly!
    }
}

// ✅ Remove default or add sleep
for {
    select {
    case msg := <-ch:
        process(msg)
    }
}

### 3. Break Confusion

In [ ]:
// Break only exits select
for {
    select {
    case <-ch:
        break // Doesn't exit loop!
    }
}

// ✅ Use labeled break
loop:
for {
    select {
    case <-ch:
        break loop
    }
}

## Quick Reference

In [ ]:
// Basic receive
select {
case v := <-ch:
    // Handle v
}

// Multiple channels
select {
case v1 := <-ch1:
    // Handle v1
case v2 := <-ch2:
    // Handle v2
}

// Non-blocking
select {
case v := <-ch:
    // Handle v
default:
    // No value ready
}

// Timeout
select {
case v := <-ch:
    // Handle v
case <-time.After(1 * time.Second):
    // Timeout
}

// Send
select {
case ch <- value:
    // Sent
default:
    // Channel full/closed
}